# Начальные модули

In [9]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import torch
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [10]:
#подключение ГПУ
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#для репитативности результатов
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)

cuda


In [11]:
#отключение предупреждений
import warnings 
warnings.filterwarnings('ignore')

**считали ключи (название файла - бактерия)**


In [12]:
profiles_ID = pd.read_csv("MS_profiles/sets/profiles_group_ID.csv", sep=';', header=None)
profiles_ID

,0,1,2
0,e0ccca6e-5d79-4c53-b1b8-d80d444e7585,Bacillus_licheniformis,KUskv2(1)
1,b3e3775b-3309-481a-ad5b-74589da92796,Bacillus_licheniformis,KUskv2(1)
2,ff9bee2a-2a61-4ece-a65b-a9c91f6cd942,Bacillus_licheniformis,KUskv2(1)
3,07441fb6-9e0e-4e9e-b9be-7a3f5cf1046c,Bacillus_licheniformis,KU16oc(1)2011
4,566e6be2-053b-45a7-91ea-3818c6d2f599,Bacillus_licheniformis,KU16oc(1)2011
...,...,...,...
144,006bcbe8-8ac5-4939-a9c8-73ba3638b7bb,Bacillus_atrophaeus,47(11)il
145,f1dc0276-8db7-48a3-a82a-b96d56b5a96c,Bacillus_atrophaeus,47(8)il
146,af8e46eb-ce25-4069-81b8-cc9d04307819,Bacillus_atrophaeus,47(8)il
147,16379977-b2cb-4907-9460-680c75c15906,Bacillus_chungangenis,45(3)il


In [13]:
t = pd.read_csv("MS_profiles/0a2c8dcb-9e19-47a4-9f35-0d7a5728eaa7", sep=';', index_col=0, header=None)
t = t.T
t[15001.0] = 0

# пустой словарь для удобного считывания 
MS_profiles=pd.DataFrame({k: pd.Series(dtype=float) for k in t.columns})

#считываем каждый файл
for i in profiles_ID.index:
    s = pd.read_csv("MS_profiles/"+profiles_ID.at[i, 0], sep=';', index_col=0, header=None)
    s = s.T
    s[15001.0] = profiles_ID.at[i, 1]
    s[15002.0] = profiles_ID.at[i, 2]
    MS_profiles = MS_profiles.append(s)

#делаем красивые индексы
MS_profiles.index = profiles_ID.index
MS_profiles

,3000.0,3001.0,3002.0,3003.0,3004.0,3005.0,3006.0,3007.0,3008.0,3009.0,...,14993.0,14994.0,14995.0,14996.0,14997.0,14998.0,14999.0,15000.0,15001.0,15002.0
0,0.0,0.006989,0.009474,0.011865,0.014919,0.013818,0.014891,0.014112,0.011084,0.013803,...,0.002440,0.002064,0.001836,0.001591,0.001053,0.000658,0.000551,0.0,Bacillus_licheniformis,KUskv2(1)
1,0.0,0.004705,0.005630,0.006658,0.007763,0.007013,0.005352,0.005495,0.008364,0.010875,...,0.003029,0.002438,0.001868,0.001774,0.001079,0.000677,0.000545,0.0,Bacillus_licheniformis,KUskv2(1)
2,0.0,0.004297,0.005857,0.006299,0.005749,0.005669,0.006205,0.005808,0.007874,0.010852,...,0.000879,0.000885,0.000811,0.000625,0.000871,0.000786,0.000364,0.0,Bacillus_licheniformis,KUskv2(1)
3,0.0,0.008850,0.014245,0.018715,0.022316,0.027317,0.035863,0.039533,0.033606,0.027822,...,0.003156,0.002286,0.001744,0.001688,0.001384,0.000959,0.000605,0.0,Bacillus_licheniformis,KU16oc(1)2011
4,0.0,0.003600,0.010289,0.010546,0.010758,0.014952,0.012372,0.011998,0.013489,0.008498,...,0.001187,0.000998,0.000950,0.001003,0.000939,0.000655,0.000255,0.0,Bacillus_licheniformis,KU16oc(1)2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,0.0,0.002941,0.001378,0.003481,0.005927,0.006573,0.008819,0.009778,0.009787,0.010087,...,0.001064,0.001024,0.000923,0.000894,0.000749,0.000473,0.000252,0.0,Bacillus_atrophaeus,47(11)il
145,0.0,0.004210,0.006997,0.009099,0.007007,0.004293,0.003677,0.002448,0.002965,0.003225,...,0.001017,0.001101,0.000811,0.000429,0.000261,0.000131,0.000261,0.0,Bacillus_atrophaeus,47(8)il
146,0.0,0.030378,0.036577,0.031172,0.029491,0.023378,0.025313,0.024803,0.014782,0.009577,...,0.000300,0.000667,0.000930,0.001027,0.001127,0.000947,0.000256,0.0,Bacillus_atrophaeus,47(8)il
147,0.0,0.004619,0.004678,0.004688,0.004471,0.002587,0.002639,0.003947,0.006853,0.008931,...,0.001599,0.001406,0.000913,0.000516,0.000499,0.000422,0.000263,0.0,Bacillus_chungangenis,45(3)il


In [39]:
MS_profiles = MS_profiles.transpose()
copy = MS_profiles.copy().to_numpy()
copy1 = copy[12001:, :]
copy = np.vstack((copy1, copy))
copy = pd.DataFrame(copy[:12003, :], index = np.hstack((np.array(['group', 'ID']),\
                  MS_profiles.index.to_numpy()[:12001])),\
                    columns = MS_profiles.columns)
copy

[[0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0069892666 0.0047051591 0.0042966269 ... 0.0303781999 0.0046188716
  0.0042059866]
 [0.0094740155 0.0056303437 0.0058573304 ... 0.0365769111 0.0046778588
  0.0062443139]
 ...
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 ['Bacillus_licheniformis' 'Bacillus_licheniformis'
  'Bacillus_licheniformis' ... 'Bacillus_atrophaeus'
  'Bacillus_chungangenis' 'Bacillus_chungangenis']
 ['KUskv2(1)' 'KUskv2(1)' 'KUskv2(1)' ... '47(8)il' '45(3)il' '45(3)il']]
[['Bacillus_licheniformis' 'Bacillus_licheniformis'
  'Bacillus_licheniformis' 'Bacillus_licheniformis'
  'Bacillus_licheniformis' 'Bacillus_licheniformis'
  'Bacillus_licheniformis' 'Bacillus_licheniformis'
  'Bacillus_licheniformis' 'Bacillus_licheniformis'
  'Bacillus_licheniformis' 'Bacillus_licheniformis'
  'Bacillus_licheniformis' 'Bacillus_licheniformis'
  'Bacillus_licheniformis' 'Bacillus_licheniformis'
  'Bacillus_licheniformis' 'Bacillus_licheniformis'
  'Bacillus_licheniformis' 'Bacillus_licheniformis'
  'Bacill

,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,148
group,Bacillus_licheniformis,Bacillus_licheniformis,Bacillus_licheniformis,Bacillus_licheniformis,Bacillus_licheniformis,Bacillus_licheniformis,Bacillus_licheniformis,Bacillus_licheniformis,Bacillus_licheniformis,Bacillus_licheniformis,...,Anoxybacillus_flavithermus,Anoxybacillus_flavithermus,E-Coli,E-Coli,Bacillus_atrophaeus,Bacillus_atrophaeus,Bacillus_atrophaeus,Bacillus_atrophaeus,Bacillus_chungangenis,Bacillus_chungangenis
ID,KUskv2(1),KUskv2(1),KUskv2(1),KU16oc(1)2011,KU16oc(1)2011,KU16oc(3)2011,KU16oc(3)2011,KG16(2011)1_(k14dt)-fs,KG16(2011)1_(k14dt)-fs,2U,...,Seya,Seya,E-Coli,E-Coli,47(11)il,47(11)il,47(8)il,47(8)il,45(3)il,45(3)il
3000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3001.0,0.006989,0.004705,0.004297,0.00885,0.0036,0.010804,0.003208,0.00652,0.003026,0.014098,...,0.003142,0.003764,0.004722,0.0,0.000884,0.002941,0.00421,0.030378,0.004619,0.004206
3002.0,0.009474,0.00563,0.005857,0.014245,0.010289,0.019208,0.005796,0.007808,0.0,0.015479,...,0.004696,0.005216,0.001943,0.000363,0.003137,0.001378,0.006997,0.036577,0.004678,0.006244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14996.0,0.001591,0.001774,0.000625,0.001688,0.001003,0.003931,0.001176,0.001511,0.000294,0.001952,...,0.00152,0.001152,0.000465,0.0,0.000631,0.000894,0.000429,0.001027,0.000516,0.001193
14997.0,0.001053,0.001079,0.000871,0.001384,0.000939,0.003283,0.001687,0.001311,0.000192,0.001617,...,0.001672,0.000851,0.000457,0.000013,0.000402,0.000749,0.000261,0.001127,0.000499,0.000849
14998.0,0.000658,0.000677,0.000786,0.000959,0.000655,0.001761,0.001108,0.000841,0.0,0.000959,...,0.000887,0.000543,0.000329,0.00003,0.000198,0.000473,0.000131,0.000947,0.000422,0.000692
14999.0,0.000551,0.000545,0.000364,0.000605,0.000255,0.000599,0.000461,0.000439,0.000205,0.000348,...,0.000462,0.000265,0.000115,0.00007,0.000145,0.000252,0.000261,0.000256,0.000263,0.00034


In [9]:
for k in tqdm(range(200, 1000, 100)):
    ## inference noise

    noise_factor = k/100
    for n, i in tqdm (enumerate(pd.unique(MS_profiles[15002.0]))):
        s = MS_profiles.loc[MS_profiles[15002.0] == i].copy()
        s.index = {j for j in range (len(s.index))}#красивые индексы   
        s = s.sample()
        s = s.to_numpy()[0]
        main = s.copy()
        amount = 3
        for m, j in enumerate(range(amount)):
            s = main.copy()
            tmp = s[:12001].astype(float)
            tmp = tmp +\
            np.random.normal(loc=0.0, scale = noise_factor*tmp, size = \
            (len(MS_profiles.columns) - 2))#шумим
            tmp = abs(tmp)
            s[:12001] = tmp
            np.place(s[:12001], s[:12001] > 1, 1)
            if (m == 0)and(n == 0):
                S = np.array([s])
            else:    
                S = np.append(S, np.array([s]), axis = 0)
    MS_profiles_inference = pd.DataFrame(S, columns = MS_profiles.columns)
    MS_profiles_inference.index = {i for i in range(len(MS_profiles_inference.index))}

    MS_profiles_inference.to_csv(r"MS_profiles\sets\MS_profiles_sintes_norm_noise_"+str(k)+"%_inference_2", sep=';', header=True, index=True,)



  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]